In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import numpy

from datetime import datetime
from scipy import stats

In [2]:
con = sqlite3.connect(r"C:\Users\Mauro\OneDrive - АНО ВО Университет Иннополис\Документы\Innopolis\Empirical Methods\DataPrimaryProject\user10.sqlite")

In [3]:
signals_data = pd.read_sql_query("""SELECT * FROM Signals;""", con)
images_data = pd.read_sql_query("""SELECT * FROM Images;""", con)

In [4]:
filter_data = images_data.loc[(images_data["state"] == "Present") | (images_data["state"] == "Absent")]

filter_data["dateTime"] = pd.to_datetime(filter_data["dateTime"])

sig_ti = signals_data.dateTime

<ipython-input-4-9e2eea3a92a1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_data["dateTime"] = pd.to_datetime(filter_data["dateTime"])


In [5]:
newfd = filter_data.filter(['id','dateTime','state'], axis=1)

newfd = newfd.reset_index(drop=True)

In [6]:
newfd

,id,dateTime,state
0,3,2021-03-30 13:00:50.296966,Absent
1,5,2021-03-30 13:00:51.091863,Present
2,7,2021-03-30 13:01:17.452682,Present
3,9,2021-03-30 13:01:43.328120,Present
4,11,2021-03-30 13:01:58.903678,Present
...,...,...,...
887,1755,2021-05-14 17:36:30.014077,Present
888,1757,2021-05-14 17:36:40.606190,Present
889,1759,2021-05-14 17:36:42.957721,Present
890,1761,2021-05-14 17:38:10.272145,Present


In [7]:
pstates = list()
astates = list()

states = newfd.groupby([(newfd.state != newfd.state.shift()).cumsum()]) 

for i, g in states:
    if (len(g.dateTime.tolist()) > 1)  &  (g.state.all() == 'Present'):
        pstates.append((newfd.dateTime.iloc[g.index[0]],newfd.dateTime.iloc[g.index[0]+len(g)-1]))
    elif (g.state.all() == 'Absent') & (i > 1) & (i < len(states)-1):
        astates.append((newfd.dateTime.iloc[g.index[0]-1],newfd.dateTime.iloc[g.index[0]+len(g)]))

In [8]:
pre_sig = list()                                                             # Duration of the signals of the Present intervals

for m in range (0,len(pstates)):
    pre_intervals = list()
    for n in sig_ti:
        if (str(pstates[m][0]) <= n) &  (n <= str(pstates[m][1])):
           pre_intervals.append(n) 
    if len(pre_intervals) > 1:
       inter = pd.Series(pre_intervals, dtype = 'object')
       inter = pd.to_datetime(inter)
    pre_sig.append(inter.iloc[-1] - inter.iloc[0])

In [9]:
abs_sig = list()                                                             # Duration of the signals of the Present intervals

for m in range (0,len(astates)):
    abs_intervals = list()
    for n in sig_ti:
        if (str(astates[m][0]) <= n) &  (n <= str(astates[m][1])):
           abs_intervals.append(n) 
    if len(abs_intervals) > 1:
       inter = pd.Series(abs_intervals, dtype = 'object')
       inter = pd.to_datetime(inter)
    abs_sig.append(inter.iloc[-1] - inter.iloc[0])

In [10]:
a = pre_sig
b = abs_sig

In [11]:
a = [p.total_seconds()/60 for p in pre_sig ]                          # Time in seconds
a = pd.DataFrame(a)

b = [a.total_seconds()/60 for a in abs_sig ]                          # Time in seconds
b =pd.DataFrame(b)

alpha = 0.05

Q_1_a = a.quantile(0.25)
Q_3_a = a.quantile(0.75)
Iqr_a  = Q_3_a - Q_1_a

Q_1_b = b.quantile(0.25)
Q_3_b = b.quantile(0.75)
Iqr_b = Q_3_b - Q_1_b

In [12]:
filter_a = a[((Q_1_a - 1.5 * Iqr_a) <= a) | (a <=  (Q_3_a + 1.5 * Iqr_a))]
print(len(filter_a)/len(a))
filter_a = numpy.squeeze(filter_a.to_numpy())

filter_b = b[((Q_1_b - 1.5 * Iqr_b) <= b) | (b <=  (Q_3_b + 1.5 * Iqr_b))]
print(len(filter_b)/len(b))
filter_b = numpy.squeeze(filter_b.to_numpy())

1.0
1.0


In [13]:
type(filter_a)

numpy.ndarray

In [14]:
dataframe_a = pd.DataFrame(data=filter_a, columns=["Seconds"])
dataframe_a["State"] = "Present"

dataframe_b = pd.DataFrame(data=filter_b, columns=["Seconds"])
dataframe_b["State"] = "Absent"

In [15]:
data = dataframe_a.append(dataframe_b)

In [16]:
import scipy.stats as stats

stats.f_oneway(data['Seconds'][data['State'] == 'Present'] , data['Seconds'][data['State'] == 'Absent'])

F_onewayResult(statistic=5.034415542880428, pvalue=0.026170709956357667)

In [17]:
fitted_a, fitted_la = stats.boxcox(filter_a)

print(fitted_la)

k_p, p_p = stats.normaltest(filter_a)

print("p = {:g}".format(p_p))
if p_p < alpha:                                        # null hypothesis: Present Interval comes from a normal distribution
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")
    

k_p, p_p = stats.normaltest(fitted_a)

print("p = {:g}".format(p_p))
if p_p < alpha:                                        # null hypothesis: Present Interval comes from a normal distribution
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")

-0.16875378328520532
p = 5.6792e-21
The null hypothesis can be rejected
p = 0.563463
The null hypothesis cannot be rejected


In [18]:
fitted_b, fitted_lb = stats.boxcox(filter_b)

print(fitted_lb)

k_a, p_a = stats.normaltest(filter_b)

print("p = {:g}".format(p_a))
if p_a < alpha:                                         # null hypothesis: Absent Interval comes from a normal distribution
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")
    

k_a, p_a = stats.normaltest(fitted_b)

print("p = {:g}".format(p_a))
if p_a < alpha:                                         # null hypothesis: Absent Interval comes from a normal distribution
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")

0.1127277427510776
p = 3.25466e-25
The null hypothesis can be rejected
p = 0.250439
The null hypothesis cannot be rejected
